In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import numpy

from datetime import datetime
from scipy import stats

In [2]:
con = sqlite3.connect(r"C:\Users\Mauro\OneDrive - АНО ВО Университет Иннополис\Документы\Innopolis\Empirical Methods\DataPrimaryProject\user1.sqlite")

In [3]:
signals_data = pd.read_sql_query("""SELECT * FROM Signals;""", con)
images_data = pd.read_sql_query("""SELECT * FROM Images;""", con)

In [4]:
filter_data = images_data.loc[(images_data["state"] == "Present") | (images_data["state"] == "Absent")]

filter_data["dateTime"] = pd.to_datetime(filter_data["dateTime"])

sig_ti = signals_data.dateTime

<ipython-input-4-9e2eea3a92a1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_data["dateTime"] = pd.to_datetime(filter_data["dateTime"])


In [5]:
newfd = filter_data.filter(['id','dateTime','state'], axis=1)

newfd = newfd.reset_index(drop=True)

In [6]:
newfd

,id,dateTime,state
0,1,2021-03-30 13:13:55.899572,Absent
1,2,2021-03-30 13:13:57.749004,Present
2,4,2021-03-30 13:25:45.179473,Present
3,5,2021-03-30 13:25:47.124525,Absent
4,6,2021-03-30 13:25:47.511522,Present
...,...,...,...
1309,2514,2021-04-27 17:52:31.835608,Present
1310,2516,2021-04-27 17:54:50.824965,Present
1311,2518,2021-04-27 17:55:03.524461,Present
1312,2520,2021-04-27 17:55:46.828018,Present


In [7]:
pstates = list()
astates = list()

states = newfd.groupby([(newfd.state != newfd.state.shift()).cumsum()]) 

for i, g in states:
    if (len(g.dateTime.tolist()) > 1)  &  (g.state.all() == 'Present'):
        pstates.append((newfd.dateTime.iloc[g.index[0]],newfd.dateTime.iloc[g.index[0]+len(g)-1]))
    elif (g.state.all() == 'Absent') & (i > 1) & (i < len(states)-1):
        astates.append((newfd.dateTime.iloc[g.index[0]-1],newfd.dateTime.iloc[g.index[0]+len(g)]))

In [8]:
pre_sig = list()                                                             # Duration of the signals of the Present intervals

for m in range (0,len(pstates)):
    pre_intervals = list()
    for n in sig_ti:
        if (str(pstates[m][0]) <= n) &  (n <= str(pstates[m][1])):
           pre_intervals.append(n) 
    if len(pre_intervals) > 1:
       inter = pd.Series(pre_intervals, dtype = 'object')
       inter = pd.to_datetime(inter)
    pre_sig.append(inter.iloc[-1] - inter.iloc[0])

In [9]:
abs_sig = list()                                                             # Duration of the signals of the Present intervals

for m in range (0,len(astates)):
    abs_intervals = list()
    for n in sig_ti:
        if (str(astates[m][0]) <= n) &  (n <= str(astates[m][1])):
           abs_intervals.append(n) 
    if len(abs_intervals) > 1:
       inter = pd.Series(abs_intervals, dtype = 'object')
       inter = pd.to_datetime(inter)
    abs_sig.append(inter.iloc[-1] - inter.iloc[0])

In [10]:
a = pre_sig
b = abs_sig

In [11]:
a = [p.total_seconds()/60 for p in pre_sig ]                          # Time in seconds
a = pd.DataFrame(a)

b = [a.total_seconds()/60 for a in abs_sig ]                          # Time in seconds
b =pd.DataFrame(b)

alpha = 0.05

Q_1_a = a.quantile(0.25)
Q_3_a = a.quantile(0.75)
Iqr_a  = Q_3_a - Q_1_a

Q_1_b = b.quantile(0.25)
Q_3_b = b.quantile(0.75)
Iqr_b = Q_3_b - Q_1_b

In [12]:
filter_a = a[((Q_1_a - 1.5 * Iqr_a) <= a) | (a <=  (Q_3_a + 1.5 * Iqr_a))]
print(len(filter_a)/len(a))
filter_a = numpy.squeeze(filter_a.to_numpy())

filter_b = b[((Q_1_b - 1.5 * Iqr_b) <= b) | (b <=  (Q_3_b + 1.5 * Iqr_b))]
print(len(filter_b)/len(b))
filter_b = numpy.squeeze(filter_b.to_numpy())

1.0
1.0


In [13]:
fitted_a, fitted_la = stats.boxcox(filter_a)

print(fitted_la)

k_p, p_p = stats.normaltest(filter_a)

print("p = {:g}".format(p_p))
if p_p < alpha:                                        # null hypothesis: Present Interval comes from a normal distribution
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")
    

k_p, p_p = stats.normaltest(fitted_a)

print("p = {:g}".format(p_p))
if p_p < alpha:                                        # null hypothesis: Present Interval comes from a normal distribution
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")

-0.002249959461091528
p = 2.4625e-20
The null hypothesis can be rejected
p = 0.0640534
The null hypothesis cannot be rejected


In [14]:
fitted_b, fitted_lb = stats.boxcox(filter_b)

print(fitted_lb)

k_a, p_a = stats.normaltest(filter_b)

print("p = {:g}".format(p_a))
if p_a < alpha:                                         # null hypothesis: Absent Interval comes from a normal distribution
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")
    

k_a, p_a = stats.normaltest(fitted_b)

print("p = {:g}".format(p_a))
if p_a < alpha:                                         # null hypothesis: Absent Interval comes from a normal distribution
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")

0.10867070744628195
p = 1.30304e-65
The null hypothesis can be rejected
p = 0.0491978
The null hypothesis can be rejected
